In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [2]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover, NGram
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [3]:
df_sentences = spark.createDataFrame([(0, 'Hi i heard about Spark'),
                                      (1, 'I wish java could use case classes'),
                                      (2, 'Logistic,regression,models,are,neat')],
                                     ['id', 'sentence'])
df_sentences.printSchema()

root
 |-- id: long (nullable = true)
 |-- sentence: string (nullable = true)



In [4]:
df_sentences.show(truncate=False)

+---+-----------------------------------+
|id |sentence                           |
+---+-----------------------------------+
|0  |Hi i heard about Spark             |
|1  |I wish java could use case classes |
|2  |Logistic,regression,models,are,neat|
+---+-----------------------------------+



In [5]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

In [6]:
regex_tokenizer = RegexTokenizer(inputCol='sentence', outputCol='words', pattern='\\W')

In [7]:
count_tokens = udf(lambda words:len(words), IntegerType())

In [8]:
tokenized = tokenizer.transform(df_sentences)

In [9]:
tokenized.show()

+---+--------------------+--------------------+
| id|            sentence|               words|
+---+--------------------+--------------------+
|  0|Hi i heard about ...|[hi, i, heard, ab...|
|  1|I wish java could...|[i, wish, java, c...|
|  2|Logistic,regressi...|[logistic,regress...|
+---+--------------------+--------------------+



In [10]:
tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi i heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic,regress...|     1|
+---+--------------------+--------------------+------+



In [11]:
rg_tokenized = regex_tokenizer.transform(df_sentences)

In [12]:
rg_tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi i heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic, regres...|     5|
+---+--------------------+--------------------+------+



In [13]:
sentenceDataFrame = spark.createDataFrame([(0, ['I', 'saw', 'the', 'green', 'horse']),
                                           (1, ['Mary', 'had', 'a', 'little', 'lamb'])], 
                                          ['id', 'tokens'])

In [14]:
sentenceDataFrame.show()

+---+--------------------+
| id|              tokens|
+---+--------------------+
|  0|[I, saw, the, gre...|
|  1|[Mary, had, a, li...|
+---+--------------------+



In [15]:
remover = StopWordsRemover(inputCol='tokens', outputCol='filtered')

In [16]:
remover.transform(sentenceDataFrame).show()

+---+--------------------+--------------------+
| id|              tokens|            filtered|
+---+--------------------+--------------------+
|  0|[I, saw, the, gre...| [saw, green, horse]|
|  1|[Mary, had, a, li...|[Mary, little, lamb]|
+---+--------------------+--------------------+



In [17]:
wordDataFrame = spark.createDataFrame([(0, ["Hi", "i", "heard", "about", "Spark"]),
                                       (1, ["I", "wish", "java", "could", "use", "case", "classes"]),
                                       (2, ["Logistic", "regression", "models", "are", "neat"])],
                                      ["id", "words"])

In [18]:
ngram = NGram(n=2, inputCol='words', outputCol='grams')

In [19]:
ngram.transform(wordDataFrame).show(truncate=False)

+---+------------------------------------------+------------------------------------------------------------------+
|id |words                                     |grams                                                             |
+---+------------------------------------------+------------------------------------------------------------------+
|0  |[Hi, i, heard, about, Spark]              |[Hi i, i heard, heard about, about Spark]                         |
|1  |[I, wish, java, could, use, case, classes]|[I wish, wish java, java could, could use, use case, case classes]|
|2  |[Logistic, regression, models, are, neat] |[Logistic regression, regression models, models are, are neat]    |
+---+------------------------------------------+------------------------------------------------------------------+

